# 03. Criação da Camada Gold

Cria as tabelas agregadas e métricas de negócio para consumo no Power BI.

In [0]:
from pyspark.sql.functions import col, round, row_number, expr, when
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F

In [0]:
silver_df = spark.table("workspace.nba_lakehouse_silver.fato_ranking_fantasy")
dim_players_silver_df = spark.table("workspace.nba_lakehouse_silver.dim_jogadores_silver")

print("Tabelas 'fato_ranking_fantasy' e 'dim_jogadores_silver' carregadas.")

### Cálculo de Métricas de Negócio

In [0]:
def create_gold_metrics(df):

    df_casted = df.withColumn("Projected_PTS", col("Projected_PTS").cast(DoubleType())) \
                  .withColumn("Projected_TRB", col("Projected_TRB").cast(DoubleType())) \
                  .withColumn("Projected_AST", col("Projected_AST").cast(DoubleType())) \
                  .withColumn("Projected_STL", col("Projected_STL").cast(DoubleType())) \
                  .withColumn("Projected_BLK", col("Projected_BLK").cast(DoubleType())) \
                  .withColumn("Projected_TOV", col("Projected_TOV").cast(DoubleType()))

    fantasy_df = df_casted.withColumn(
        "Fantasy_Score_Projected",
        round(
            (col("Projected_PTS") * 1) +
            (col("Projected_TRB") * 1.2) +
            (col("Projected_AST") * 1.5) +
            (col("Projected_STL") * 3) +
            (col("Projected_BLK") * 3) -
            (col("Projected_TOV") * 1),
            2
        )
    )

    window_spec = Window.orderBy(col("Fantasy_Score_Projected").desc())

    analysis_df = fantasy_df.withColumn("Fantasy_Rank", row_number().over(window_spec)) \
                            .withColumn("Value_vs_ADP", col("ADP_Rank") - col("Fantasy_Rank"))
    
    return analysis_df

In [0]:
gold_df = create_gold_metrics(silver_df)

### Tabela Gold Fato 1: `fato_ranking_fantasy`

In [0]:
fato_ranking_fantasy_gold_df = gold_df.select(
    "Player_Key",
    "Pos",
    "G",
    "Past_Season_PTS",
    "Past_Season_TRB",
    "Past_Season_AST",
    "Past_Season_STL",
    "Past_Season_BLK",
    "Past_Season_TOV",
    "Projected_PTS",
    "Projected_TRB",
    "Projected_AST",
    "Projected_STL",
    "Projected_BLK",
    "Projected_TOV",
    "ADP_Rank",
    "ADP_Avg",
    "Fantasy_Score_Projected",
    "Fantasy_Rank",
    "Value_vs_ADP"
)

### Tabela Gold Fato 2: `player_metrics_unpivoted`

In [0]:
unpivot_expr_past = "stack(6, 'PTS', Past_Season_PTS, 'TRB', Past_Season_TRB, 'AST', Past_Season_AST, 'STL', Past_Season_STL, 'BLK', Past_Season_BLK, 'TOV', Past_Season_TOV) as (Stat_Category, Value)"

past_df_unpivoted = gold_df.select("Player_Key", "Player", expr(unpivot_expr_past)) \
                           .withColumn("Metric_Type", expr("'Past Season'"))

unpivot_expr_proj = "stack(6, 'PTS', Projected_PTS, 'TRB', Projected_TRB, 'AST', Projected_AST, 'STL', Projected_STL, 'BLK', Projected_BLK, 'TOV', Projected_TOV) as (Stat_Category, Value)"

proj_df_unpivoted = gold_df.select("Player_Key", "Player", expr(unpivot_expr_proj)) \
                           .withColumn("Metric_Type", expr("'Projected'"))

player_metrics_unpivoted_df = past_df_unpivoted.unionByName(proj_df_unpivoted)

### Tabela Gold Dimensão: `dim_jogadores`

In [0]:
metrics_for_dim = gold_df.select(
    "Player_Key",
    "G",
    "Fantasy_Score_Projected",
    "Fantasy_Rank",
    "Value_vs_ADP"
)

dim_jogadores_gold_df = dim_players_silver_df.join(
    metrics_for_dim,
    "Player_Key",
    "left"
).withColumn(
    "Risk_Level",
    when(col("G").isNull(), "Alto")
    .when(col("G") < 41, "Alto")
    .when(col("G") < 65, "Médio")
    .otherwise("Baixo")
).select(
    dim_players_silver_df.Player_Key,
    dim_players_silver_df.Player_ID_NBA,
    col("Player_Name").alias("Player"),
    dim_players_silver_df.Image_URL,
    dim_players_silver_df.Flag_URL,
    col("Position").alias("Pos"),
    dim_players_silver_df.Height_Str,
    dim_players_silver_df.Weight_kg,
    dim_players_silver_df.Birthdate,
    dim_players_silver_df.Country,
    dim_players_silver_df.School,
    dim_players_silver_df.Draft_Year,
    col("G").alias("Last_Season_Played_Games"),
    col("Risk_Level"),
    metrics_for_dim.Fantasy_Score_Projected,
    metrics_for_dim.Fantasy_Rank,
    metrics_for_dim.Value_vs_ADP
).orderBy(col("Fantasy_Rank").asc_nulls_last())

In [0]:
display(dim_jogadores_gold_df.limit(20))

### Exportação para Camada Gold

In [0]:
def save_dataframe_as_table(df, table_name, mode="overwrite"):
    (df.write.mode(mode)
        .format("delta")
        .option("overwriteSchema", "true")
        .saveAsTable(table_name))
    
    print(f"Tabela '{table_name}' salva com sucesso.")

In [0]:
save_dataframe_as_table(fato_ranking_fantasy_gold_df, "workspace.nba_lakehouse_gold.fato_ranking_fantasy")
save_dataframe_as_table(player_metrics_unpivoted_df, "workspace.nba_lakehouse_gold.player_metrics_unpivoted")
save_dataframe_as_table(dim_jogadores_gold_df, "workspace.nba_lakehouse_gold.dim_jogadores")

print("\n--- Processo da Camada Gold Concluído. Todas as tabelas foram salvas. ---")